<div class="alert alert-success" role="quote" style="text-align:center;font-size:120%;;">
  In the name of Allah, the most Merciful and the most Beneficent!</div>

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")
prices.shape

In [ ]:
prices.head()

In [ ]:
adjustment = prices.AdjustmentFactor.value_counts()
adjustment.plot(kind="pie")


In [ ]:
expect = prices.ExpectedDividend.value_counts()
expect.plot(kind="pie")

In [ ]:
import missingno
missingno.matrix(prices)

In [ ]:
flag = prices.SupervisionFlag.value_counts()
flag.plot(kind="pie")

In [ ]:
unnecessaries = ["RowId", "SupervisionFlag", "ExpectedDividend", "AdjustmentFactor"]
prices.drop(unnecessaries, axis=1, inplace=True)

In [ ]:
industries = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv")
industries.head(5)

In [ ]:
missingno.matrix(industries);

In [ ]:
industries['33SectorName'].value_counts().plot(kind="pie", figsize=(16,12))

In [ ]:
industries.isnull().sum()

In [ ]:
industries['Section/Products']

In [ ]:
df = pd.merge(prices, industries[['33SectorName', "SecuritiesCode"]], how="left", on="SecuritiesCode")

In [ ]:
df

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['33SectorName'] = encoder.fit_transform(df['33SectorName'])

In [ ]:
df.Date = pd.to_datetime(df.Date)

# First for checking drop some columns from X

In [ ]:
def date_to_classic(df):
    dt = df.Date
    df['Year'] = dt.dt.year
    df['Month'] = dt.dt.month
    df['Week'] = dt.dt.week
    df['quarter'] = dt.dt.quarter
    df.drop("Date", axis=1, inplace=True)

In [ ]:
date_to_classic(df)

In [ ]:
df = df.dropna()

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

train=df[df.Year<2021]
val=df[df.Year>=2021]
X_train = train
Y_train = X_train.pop("Target")
X_val = val
Y_val = X_val.pop("Target")

In [ ]:
params = {
            'num_leaves': 2565,
            'n_estimators': 621,
            'max_bin': 95,
            'learning_rate': 0.6818202991034834,
    }

model = LGBMRegressor(**params)

In [ ]:
model.fit(X_train,Y_train)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

Y_pred = model.predict(X_val)
mean_squared_error(Y_val, Y_pred)

In [ ]:
le = LabelEncoder()
X['SimpleCode'] = le.fit_transform(X.SecuritiesCode)

In [ ]:
X.drop("SecuritiesCode", axis=1, inplace=True)

In [ ]:
cat_names = ['SecuritiesCode', '33SectorName']


In [ ]:
X_train

In [ ]:
lgb_train = lgb.Dataset(X_train, Y_train, categorical_feature=None)
lgb_eval  = lgb.Dataset(X_val, Y_val, categorical_feature=None, reference=lgb_train)
params = {
    'objective': 'mse',
    'metric': 'rmse',
    'num_leaves': 255,
    'learning_rate': 0.005,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 5,
    'seed': 1,
    'verbose': 1,
    'force_row_wise' : True
}
evals_result = {}
gbm = lgb.train(
        params, 
        lgb_train,
        num_boost_round = 4000,
        valid_sets = (lgb_train, lgb_eval), 
        feature_name = X_train.columns.tolist(),
        categorical_feature = cat_names,
        verbose_eval = 100, 
        evals_result = evals_result,
        early_stopping_rounds = 100)

In [ ]:
import lightgbm

gbm

In [ ]:
env = jpx_tokyo_market_prediction.make_env()

In [ ]:
iter_test = env.iter_test()

In [ ]:
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    ds=[prices, options, financials, trades, secondary_prices, sample_prediction]
    df = sample_prediction[["Date","SecuritiesCode"]]
    df["High"] = prices["High"]
    df["Open"] = prices["Open"]
    df["Close"] = prices["Close"]
    df["Low"] = prices["Low"]
    df.Date = pd.to_datetime(df.Date)
    df = date_to_classic(df)
    print(df)
    sample_prediction["Prediction"] = model_o.predict(df)
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)
    #sample_prediction["Prediction"] = model_o.predict(ndf)
    #sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    #sample_prediction.Rank = np.arange(0,2000)
    #sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    #sample_prediction.drop(["Prediction"],axis=1)
    #submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    #env.predict(submission)

In [ ]:
df